In [14]:
from data_import import columns_from
import re

In [ ]:
cols = {
    header: [
        re.sub('^[^a-zA-Z"\']*', '', line)
    for cell_lines in [cell.split('\n') for cell in cells] for line in cell_lines]
for header, cells in columns_from('results_en.csv').items() }

for header, cells in cols.items():
    print(header)
    for cell in cells: print(cell)
    print()